In [4]:
import nengo
import numpy as np

f = open('Jaco-Output.txt')
lines = f.readlines()
f.close()
    

In [10]:
inputs = np.array([[float(x) for x in line.strip().split(' ')] for line in lines[::2]])
outputs = np.array([[float(x) for x in line.strip().split(' ')] for line in lines[1::2]])

In [11]:
inputs

array([[  3.80166600e+00,  -2.87334000e-01,  -2.07875000e-01, ...,
          5.20636000e-01,   1.41593500e+00,  -3.76730000e-02],
       [  3.84304900e+00,  -2.87246000e-01,  -2.07841000e-01, ...,
          5.20733000e-01,   1.41594600e+00,  -3.76730000e-02],
       [  3.87323200e+00,  -2.81564000e-01,  -2.06492000e-01, ...,
          5.28153000e-01,   1.41628400e+00,  -3.76730000e-02],
       ..., 
       [  1.54240769e+02,   8.54690000e-01,   6.99645000e-01, ...,
          1.08312750e+01,   6.73911000e-01,   8.19502000e-01],
       [  1.54280748e+02,   8.54714000e-01,   6.99680000e-01, ...,
          1.08312750e+01,   6.73911000e-01,   8.19502000e-01],
       [  1.54321485e+02,   8.54722000e-01,   6.99683000e-01, ...,
          1.08312750e+01,   6.73911000e-01,   8.19502000e-01]])